In [106]:
import random

CVR = [(1,2)]*700 + [(2,)]*770 + [(3,)]*850 + [(4,1)]*350 + [(5,1)]*300 + [(6,)]*100 # 6 = Jill Stein

def project_FPV(bal, hopeful):
    list_bal = list(bal)
    if len(list_bal) == 0:
        return 6 # Jill Stein
    while bal[0] not in hopeful:
        bal = bal[1:]
        if len(bal) == 0:
            return 6 # Jill Stein
    return bal[0]

def noise_bal(bal, cands = {1,2,3,4,5}):
    if bal == (6,):
        return random.choice([(1,2), (2,), (3,), (4,1), (5,1)])
    else:
        noise_type = random.choice([1,2,3,4,5]) 
        if noise_type == 2: # delete ranking
            if len(bal) == 1:
                noise_type = 1
            else:
                del_pos = random.randint(0, len(bal)-1)
                new_bal = list(bal)
                del new_bal[del_pos]
                return tuple(new_bal)
        if noise_type == 1:# insert ranking
            # our ballots are all short, so no need to worry about having no cands to insert
            cand_to_insert = random.choice([c for c in cands if c not in bal])
            new_bal = list(bal)
            insert_pos = random.randint(0, len(new_bal))
            new_bal.insert(insert_pos, cand_to_insert)
            return tuple(new_bal)
        if noise_type == 3: # permute two rankings
            if len(bal) == 1:
                noise_type = 4
            else:
                bal_list = list(bal)
                indices = random.sample(range(len(bal)), 2)
                bal_list[indices[0]], bal_list[indices[1]] = bal_list[indices[1]], bal_list[indices[0]]
                return tuple(bal_list)
            return tuple(sorted(set(bal).intersection(cands)))
        if noise_type == 4: #switch one ranking with another non-ranked candidate
            cand_to_switch_in = random.choice([c for c in cands if c not in bal])
            switch_pos = random.randint(0, len(bal)-1)
            bal_list = list(bal)
            bal_list[switch_pos] = cand_to_switch_in
            return tuple(bal_list)
        if noise_type == 5: # become Jill Stein
            return (6,)
    raise Exception("Should not reach here")

# sample noise% of the indices in BAL to noise
noise = 0.01
sample_indices = random.sample(range(len(CVR)), k=int(noise*len(CVR)))
BAL = [bal for bal in CVR]
for idx in sample_indices:
    BAL[idx] = noise_bal(BAL[idx])

Round 1 audit: build confidence intervals for tallies of all cands

In [90]:
import math
from typing import Iterable, Tuple, Optional

def _norm_ppf(q: float) -> float:
    # Acklam's rational approximation to Φ^{-1}(q)
    a = [-3.969683028665376e+01,  2.209460984245205e+02,
         -2.759285104469687e+02,  1.383577518672690e+02,
         -3.066479806614716e+01,  2.506628277459239e+00]
    b = [-5.447609879822406e+01,  1.615858368580409e+02,
         -1.556989798598866e+02,  6.680131188771972e+01,
         -1.328068155288572e+01]
    c = [-7.784894002430293e-03, -3.223964580411365e-01,
         -2.400758277161838e+00, -2.549732539343734e+00,
          4.374664141464968e+00,  2.938163982698783e+00]
    d = [ 7.784695709041462e-03,  3.224671290700398e-01,
          2.445134137142996e+00,  3.754408661907416e+00]
    if not (0.0 < q < 1.0):
        if q == 0.0: return -math.inf
        if q == 1.0: return  math.inf
        raise ValueError("q must be in (0,1)")
    plow, phigh = 0.02425, 0.97575
    if q < plow:
        r = math.sqrt(-2.0 * math.log(q))
        num = (((((c[0]*r + c[1])*r + c[2])*r + c[3])*r + c[4])*r + c[5])
        den = ((((d[0]*r + d[1])*r + d[2])*r + d[3])*r + 1.0)
        return -num / den
    if q > phigh:
        r = math.sqrt(-2.0 * math.log(1.0 - q))
        num = (((((c[0]*r + c[1])*r + c[2])*r + c[3])*r + c[4])*r + c[5])
        den = ((((d[0]*r + d[1])*r + d[2])*r + d[3])*r + 1.0)
        return  num / den
    r = q - 0.5
    s = r*r
    num = (((((a[0]*s + a[1])*s + a[2])*s + a[3])*s + a[4])*s + a[5]) * r
    den = (((((b[0]*s + b[1])*s + b[2])*s + b[3])*s + b[4])*s + 1.0)
    return num / den

def Wald_CI(
    sample: Iterable[float],
    N: int,
    alpha: float = 0.05,
    bounds: Optional[Tuple[float, float]] = (-1.0, 1.0),
    clip_to_bounds: bool = True,
    fallback: str = "variance_bound",   # or "serfling"
    p0_lower: Optional[float] = 1-noise,   # use if fallback="variance_bound" and you know P(B=0) >= p0_lower
    eps_var: float = 1e-12,
    return_extras: bool = False,
):
    """
    Wald + FPC CI for the mean μ of a finite population (SRSWOR).
    If sample variance is ~0, use a conservative fallback to avoid a degenerate CI.

    Parameters
    ----------
    sample : iterable of floats
        Observed B_i (fractional allowed), assumed bounded in `bounds` (default [-1,1]).
    N : int
        Finite population size.
    alpha : float
        1 - confidence level.
    bounds : (a, b) or None
        Natural parameter bounds for μ; used only for clipping.
    clip_to_bounds : bool
        Clip CI to the provided bounds.
    fallback : {"variance_bound", "serfling"}
        Strategy when sample variance ~ 0.
        - "variance_bound": use σ_max^2 to compute a nonzero half-width (tight if you know p0_lower).
        - "serfling": use Serfling's distribution-free half-width on mean for [-1,1].
    p0_lower : float in [0,1], optional
        If you can assert P(B=0) >= p0_lower, then Var(B) <= 1 - p0_lower. Used when fallback="variance_bound".
    eps_var : float
        Threshold below which we treat the sample variance as zero.
    return_extras : bool
        If True, also return a dict with diagnostics.

    Returns
    -------
    (lo, hi) or (lo, hi, extras)
    """
    x = list(sample)
    n = len(x)
    if n == 0:
        raise ValueError("Sample is empty.")
    if N <= 0:
        raise ValueError("N must be positive.")
    if n > N:
        raise ValueError("Sample size n cannot exceed population size N.")

    m = sum(x) / n
    if n >= 2:
        ssq = sum((xi - m)**2 for xi in x) / (n - 1)
    else:
        ssq = 0.0
    s = math.sqrt(max(0.0, ssq))

    # Finite-population correction
    fpc = math.sqrt(max(0.0, 1.0 - n / N))
    z = _norm_ppf(1.0 - alpha/2.0)

    # Main path: Wald + FPC
    use_fallback = s**2 < eps_var
    if not use_fallback:
        se = (s / math.sqrt(n)) * fpc
        half = z * se
    else:
        if fallback.lower() == "variance_bound":
            # σ_max^2: worst-case variance for [-1,1] is 1 (achieved by ±1 equally likely).
            sigma2_max = 1.0
            if p0_lower is not None:
                # If P(B=0) >= p0, then Var(B) <= 1 - p0
                sigma2_max = min(sigma2_max, 1.0 - p0_lower)
            se = math.sqrt(sigma2_max / n) * fpc
            half = z * se
        elif fallback.lower() == "serfling":
            # Serfling bound for mean on [a,b]; default [-1,1] => R=2
            a, b = (-1.0, 1.0) if bounds is None else bounds
            R = b - a
            factor = 1.0 - (n - 1) / N
            # two-sided: ε = R * sqrt( factor * log(2/α) / (2n) )
            half = R * math.sqrt(max(0.0, factor) * math.log(2.0/alpha) / (2.0 * n))
        else:
            raise ValueError("fallback must be 'variance_bound' or 'serfling'")

    lo, hi = m - half, m + half
    if clip_to_bounds and bounds is not None:
        a, b = bounds
        if a >= b:
            raise ValueError("bounds must satisfy a < b")
        lo, hi = max(a, lo), min(b, hi)

    if return_extras:
        return lo, hi, {
            "mean": m, "sd": s, "var": s**2, "n": n, "N": N,
            "alpha": alpha, "z": z, "fpc": fpc, "halfwidth": half,
            "used_fallback": bool(use_fallback),
            "fallback": fallback if use_fallback else None,
            "sigma2_max_used": (1.0 - p0_lower) if (use_fallback and fallback=="variance_bound" and p0_lower is not None) else (1.0 if use_fallback and fallback=="variance_bound" else None),
        }
    return lo, hi



In [165]:
# shuffle BAL and CVR in the same way
combined = list(zip(BAL, CVR))
random.shuffle(combined)
shuffled_BAL, shuffled_CVR = zip(*combined)
sample_size = 300

#build wald CIs for all 5 candidates in the first round
round1_samples = {i: [] for i in range(1, 7)} # candidate number -> list of sample projections

hopeful = {1,2,3,4,5,6} # even though 6 is not a candidate, we still want to know when an FPV goes to 6

for i in range(0, sample_size):
    bal, cvr = shuffled_BAL[i], shuffled_CVR[i]
    if bal != cvr:
        print(f"Discrepancy found in ballot {i}: BAL={bal}, CVR={cvr}")
    fpv_bal, fpv_cvr = project_FPV(bal, hopeful), project_FPV(cvr, hopeful)
    if fpv_bal == fpv_cvr:
        for i in range(1,7):
            round1_samples[i].append(0.0)
    else: # fpv_cvr just lost a vote and fpv_bal just gained a vote
        round1_samples[fpv_bal].append(1.0)
        round1_samples[fpv_cvr].append(-1.0)
        for i in range(1,7):
            if i != fpv_bal and i != fpv_cvr:
                round1_samples[i].append(0.0)

round1_CVR_tallies = {
    1: sum(1 for bal in CVR if project_FPV(bal, hopeful) == 1),
    2: sum(1 for bal in CVR if project_FPV(bal, hopeful) == 2),
    3: sum(1 for bal in CVR if project_FPV(bal, hopeful) == 3),
    4: sum(1 for bal in CVR if project_FPV(bal, hopeful) == 4),
    5: sum(1 for bal in CVR if project_FPV(bal, hopeful) == 5),
    6: sum(1 for bal in CVR if project_FPV(bal, hopeful) == 6),
}

margin_sample_45 = [round1_samples[4][i] - round1_samples[5][i] for i in range(len(round1_samples[1]))]

#Step 1 : 5 eliminated
round1_CIs = {}

for key, value in round1_samples.items():
    ci_lo, ci_hi = Wald_CI(value, N=len(BAL), bounds=(-1.0, 1.0))
    ci_lo *= len(BAL)
    ci_hi *= len(BAL)
    ci_lo += round1_CVR_tallies[key]
    ci_hi += round1_CVR_tallies[key]
    round1_CIs[key] = (ci_lo, ci_hi)
    if key != 6:
        print(f"Round 1, Candidate {key}: CI=({ci_lo:.4f}, {ci_hi:.4f})")
    
margin_lo, margin_hi = Wald_CI(margin_sample_45, N=len(BAL), bounds=(-2.0, 2.0))
margin_hi *= len(BAL)
margin_lo *= len(BAL)
print(f"Margin CI (Cand 4 - Cand 5): ({margin_lo:.4f}, {margin_hi:.4f})")
if margin_lo > -50:
    print(f"Step 1 successful according to margin audit")
if round1_CIs[4][0] > round1_CIs[5][1]:
    print(f"Step 1 successful according to candidate CIs")

round2_samples = {i: [] for i in range(1, 7)} # candidate number -> list of sample projections

hopeful = {1,2,3,4,6} # Step 2 assumes 5 is gone

for i in range(0, sample_size):
    bal, cvr = shuffled_BAL[i], shuffled_CVR[i]
    fpv_bal, fpv_cvr = project_FPV(bal, hopeful), project_FPV(cvr, hopeful)
    if fpv_bal == fpv_cvr:
        for i in range(1,7):
            round2_samples[i].append(0.0)
    else: # fpv_cvr just lost a vote and fpv_bal just gained a vote
        round2_samples[fpv_bal].append(1.0)
        round2_samples[fpv_cvr].append(-1.0)
        for i in range(1,7):
            if i != fpv_bal and i != fpv_cvr:
                round2_samples[i].append(0.0)

round2_CVR_tallies = {i: sum(1 for bal in CVR if project_FPV(bal, hopeful) == i) for i in range(1,7)}
round2_CIs = {}

for key, value in round2_samples.items():
    if key < 5: 
        ci_lo, ci_hi = Wald_CI(value, N=len(BAL), bounds=(-1.0, 1.0))
        ci_lo *= len(BAL)
        ci_hi *= len(BAL)
        ci_lo += round2_CVR_tallies[key]
        ci_hi += round2_CVR_tallies[key]
        round2_CIs[key] = (ci_lo, ci_hi)
        print(f"Round 2, Candidate {key}: CI=({ci_lo:.4f}, {ci_hi:.4f})")
    if key == 6:
        ci_lo, ci_hi = Wald_CI(value, N=len(BAL), bounds=(-1.0, 1.0))
        ci_lo *= len(BAL)
        ci_hi *= len(BAL)
        ci_lo += round2_CVR_tallies[key]
        ci_hi += round2_CVR_tallies[key]
        q_lo = (len(BAL)- ci_hi)/3+10**(-6)
        q_hi = (len(BAL)- ci_lo)/3-10**(-6)
        print(f"exhausted in round 2 CI=({ci_lo:.4f}, {ci_hi:.4f})")
        print(f"q_low = {q_lo:.4f}, q_high = {q_hi:.4f}")

# if someone has tally, it's (only) cand 1
if round2_CIs[2][1] < q_lo and round2_CIs[3][1] < q_lo and round2_CIs[4][1] < q_lo and round2_CIs[1][1] > q_lo:
    print(f"Step 2 successful: only viable winner is Candidate 1 (reality 2a is viable).")
# if someone is eliminated, it must be 4
if round2_CIs[4][1] < round2_CIs[1][0] and round2_CIs[4][1] < round2_CIs[2][0] and round2_CIs[4][1] < round2_CIs[3][0]:
    print(f"Step 2 successful: only viable loser is Candidate 4 (reality 2b is viable).")

# Step 3a: assuming 1 was elected in step 2, nobody else gets quota, and 4 is eliminated
# to build CI for k, need a CI for (g+t), and a CI for (m+1)T - t

t_sample = []
g_sample = []
T_sample = []

hopeful = {1,2,3,4,6} # still assume 5 is gone; 1 is elected, but we want to know when ballots transfer through 12
hopeful_without1 = {2,3,4,6} 

# also need to check 4 truly gets eliminated
for i in range(0, sample_size):
    bal, cvr = shuffled_BAL[i], shuffled_CVR[i]
    fpv_bal, fpv_cvr = project_FPV(bal, hopeful), project_FPV(cvr, hopeful)
    spv_bal, spv_cvr = project_FPV(bal, hopeful_without1), project_FPV(cvr, hopeful_without1)
    if fpv_bal == fpv_cvr:
        g_sample.append(0.0)
        T_sample.append(0.0)
        if fpv_bal != 1:
            t_sample.append(0.0)
            continue
        else:
            if spv_bal == spv_cvr:
                t_sample.append(0.0)
                continue
            else: 
                if spv_cvr == 6: # this was previously a t vote, now it is not
                    t_sample.append(-1.0)
                elif spv_bal == 6: # this was previously not a t vote, now it is
                    t_sample.append(1.0)
                else:
                    t_sample.append(0.0)
            continue
    else: 
        if fpv_cvr == 1: # this was previously a T vote, now it is not
            T_sample.append(-1.0)
            if spv_cvr == 6: # this was previously a t vote, now it is not
                t_sample.append(-1.0)
            else:
                t_sample.append(0.0)
        elif fpv_bal == 1: # this was previously not a T vote, now it is
            T_sample.append(1.0)
            if spv_bal == 6: # this was previously not a t vote, now it is
                t_sample.append(1.0)
            else:
                t_sample.append(0.0)
        else: # this was previously not a FPV 1 vote, and it still isn't
            T_sample.append(0.0)
            t_sample.append(0.0)
        if fpv_cvr == 6: # this was previously a g vote, now it is not
            g_sample.append(-1.0)
        elif fpv_bal == 6: # this was previously not a g vote, now it is
            g_sample.append(1.0)
        else: 
            g_sample.append(0.0)

m=2
n=3070
gt_sample = [g_sample[i] + t_sample[i] for i in range(len(g_sample))]
Tt_sample = [(m+1)*T_sample[i] - t_sample[i] for i in range(len(t_sample))]
gt_ci_lo, gt_ci_hi = Wald_CI(gt_sample, N=len(BAL), bounds=(-2.0, 2.0))
gt_ci_lo *= len(BAL)
gt_ci_hi *= len(BAL)
gt_ci_lo += 100
gt_ci_hi += 100
print(f"(g+t) CI: ({gt_ci_lo:.4f}, {gt_ci_hi:.4f})")
Tt_ci_lo, Tt_ci_hi = Wald_CI(Tt_sample, N=len(BAL), bounds=(-2.0, 2.0))
Tt_ci_lo *= len(BAL)
Tt_ci_hi *= len(BAL)
Tt_ci_lo += (m+1)*1000 - 0
Tt_ci_hi += (m+1)*1000 - 0
print(f"(m+1)T - t CI: ({Tt_ci_lo:.4f}, {Tt_ci_hi:.4f})")
num_ground = n + (m+1)*10**(-6)
num_hi = num_ground - gt_ci_lo
num_lo = num_ground - gt_ci_hi
k_hi=num_hi/Tt_ci_lo
k_lo=num_lo/Tt_ci_hi
print(f"k CI: ({k_lo:.6f}, {k_hi:.6f})")

round3a_deg0_samples = {i: [] for i in range(2, 7)} 
round3a_deg1_samples = {i: [] for i in range(2, 7)} 

for i in range(0, sample_size):
    bal, cvr = shuffled_BAL[i], shuffled_CVR[i]
    fpv_bal, fpv_cvr = project_FPV(bal, hopeful), project_FPV(cvr, hopeful)
    spv_bal, spv_cvr = project_FPV(bal, hopeful_without1), project_FPV(cvr, hopeful_without1)
    if fpv_bal == fpv_cvr:
        if fpv_bal != 1 or spv_bal == spv_cvr:
            for i in range(2,7):
                round3a_deg0_samples[i].append(0.0)
                round3a_deg1_samples[i].append(0.0)
        else: # both fpv are 1 and spv differ
            # in either case the degree 0 impact null
            for i in range(2,7):
                round3a_deg0_samples[i].append(0.0)
                # the deg 1 impact on spv_cvr is -1, on spv_bal is +1, rest are 0
                if i == spv_cvr:
                    round3a_deg1_samples[i].append(-1.0)
                elif i == spv_bal:
                    round3a_deg1_samples[i].append(1.0)
                else:
                    round3a_deg1_samples[i].append(0.0)
    else: # different fpv 
        if fpv_cvr == 1: # previously this was a deg 1 vote, now it is not
            round3a_deg1_samples[spv_cvr].append(-1.0)
            round3a_deg0_samples[fpv_bal].append(1.0)
            # rest are 0
            for i in range(2,7):
                if i != spv_cvr:
                    round3a_deg1_samples[i].append(0.0)
                if i != fpv_bal:
                    round3a_deg0_samples[i].append(0.0)
        elif fpv_bal == 1: # previously this was not a deg 1 vote, now it is
            round3a_deg1_samples[spv_bal].append(1.0)
            round3a_deg0_samples[fpv_cvr].append(-1.0)
            # rest are 0
            for i in range(2,7):
                if i != spv_bal:
                    round3a_deg1_samples[i].append(0.0)
                if i != fpv_cvr:
                    round3a_deg0_samples[i].append(0.0)
        else: # only degree 0 impact
            round3a_deg0_samples[fpv_bal].append(1.0)
            round3a_deg0_samples[fpv_cvr].append(-1.0)
            for i in range(2,7):
                round3a_deg1_samples[i].append(0.0)
                if i != fpv_bal and i != fpv_cvr:
                    round3a_deg0_samples[i].append(0.0)

# phew!
case3a_CVR_FPV = {i: sum(1 for bal in CVR if project_FPV(bal, hopeful_without1) == i) for i in range(2,7)}

conf_intervals_3a = {}
for i in [2,3,4,6]:
    deg0_ci_lo, deg0_ci_hi = Wald_CI(round3a_deg0_samples[i], N=len(BAL), bounds=(-1.0, 1.0))
    deg0_ci_lo *= len(BAL)
    deg0_ci_hi *= len(BAL)
    deg0_ci_lo += round2_CVR_tallies[i]
    deg0_ci_hi += round2_CVR_tallies[i]
    deg1_ci_lo, deg1_ci_hi = Wald_CI(round3a_deg1_samples[i], N=len(BAL), bounds=(0, 1.0)) # in this example, the deg1 effects are non-negative
    deg1_ci_lo *= len(BAL)
    deg1_ci_hi *= len(BAL)
    deg1_ci_lo += 0
    deg1_ci_hi += 0
    total_ci_lo = deg0_ci_lo + (1-k_hi)*deg1_ci_lo
    total_ci_hi = deg0_ci_hi + (1-k_lo)*deg1_ci_hi
    if i != 6: print(f"Round 3a, Candidate {i}, Total: CI=({total_ci_lo:.4f}, {total_ci_hi:.4f})")
    conf_intervals_3a[i] = (total_ci_lo, total_ci_hi)
    if i == 6:
        q_lo = (len(BAL)- total_ci_hi)/3+10**(-6)
        q_hi = (len(BAL)- total_ci_lo)/3-10**(-6)
        print(f"exhausted in round 3a CI=({total_ci_lo:.4f}, {total_ci_hi:.4f})")
        print(f"q_low = {q_lo:.4f}, q_high = {q_hi:.4f}")

if conf_intervals_3a[2][1] < q_lo and conf_intervals_3a[3][1] < q_lo and conf_intervals_3a[4][1] < conf_intervals_3a[2][0] and conf_intervals_3a[4][1] < conf_intervals_3a[3][0]:
    print(f"Step 3a successful: Nobody has quota and 4 is eliminated.")

# step 3b: assuming 4 was eliminated in step 2, nobody else gets quota, and 1 is definitely elected
# only degree 0 effects to consider!

round3b_samples = {i: [] for i in range(1, 7)}
hopeful = {1,2,3,6} # 4 gone

for i in range(0, sample_size):
    bal, cvr = shuffled_BAL[i], shuffled_CVR[i]
    fpv_bal, fpv_cvr = project_FPV(bal, hopeful), project_FPV(cvr, hopeful)
    if fpv_bal == fpv_cvr:
        for i in range(1,7):
            round3b_samples[i].append(0.0)
    else: 
        round3b_samples[fpv_bal].append(1.0)
        round3b_samples[fpv_cvr].append(-1.0)
        for i in range(1,7):
            if i != fpv_bal and i != fpv_cvr:
                round3b_samples[i].append(0.0)

case3b_CVR_FPV = {i: sum(1 for cvr in CVR if project_FPV(cvr, hopeful) == i) for i in range(1,7)}

conf_intervals_3b = {}
for i in [1,2,3,6]:
    ci_lo, ci_hi = Wald_CI(round3b_samples[i], N=len(BAL), bounds=(-1.0, 1.0))
    ci_lo *= len(BAL)
    ci_hi *= len(BAL)
    ci_lo += case3b_CVR_FPV[i]
    ci_hi += case3b_CVR_FPV[i]
    conf_intervals_3b[i] = (ci_lo, ci_hi)
    if i != 6:
        print(f"Round 3b, Candidate {i}, Total: CI=({ci_lo:.4f}, {ci_hi:.4f})")
    if i == 6:
        q_lo = (len(BAL)- ci_hi)/3+10**(-6)
        q_hi = (len(BAL)- ci_lo)/3-10**(-6)
        print(f"exhausted in round 3b CI=({ci_lo:.4f}, {ci_hi:.4f})")
        print(f"q_low = {q_lo:.4f}, q_high = {q_hi:.4f}")

if conf_intervals_3b[2][1] < q_lo and conf_intervals_3b[3][1] < q_lo and conf_intervals_3b[1][0] > q_hi:
    print(f"Step 3b successful: 1 is definitely elected and nobody else has quota.") 

# step 4: assuming 1 elected, 4 eliminated, 5 eliminated, then cand 2 beats cand 3
# need to build k again

t_sample = []
g_sample = []
T_sample = []

hopeful = {1,2,3,6} # 4 and 5 gone; 1 has a keep factor
hopeful_without1 = {2,3,6} 

for i in range(0, sample_size):
    bal, cvr = shuffled_BAL[i], shuffled_CVR[i]
    fpv_bal, fpv_cvr = project_FPV(bal, hopeful), project_FPV(cvr, hopeful)
    spv_bal, spv_cvr = project_FPV(bal, hopeful_without1), project_FPV(cvr, hopeful_without1)
    if fpv_bal == fpv_cvr:
        g_sample.append(0.0)
        T_sample.append(0.0)
        if fpv_bal != 1:
            t_sample.append(0.0)
            continue
        else:
            if spv_bal == spv_cvr:
                t_sample.append(0.0)
                continue
            else: 
                if spv_cvr == 6: # this was previously a t vote, now it is not
                    t_sample.append(-1.0)
                elif spv_bal == 6: # this was previously not a t vote, now it is
                    t_sample.append(1.0)
                else:
                    t_sample.append(0.0)
            continue
    else: 
        if fpv_cvr == 1: # this was previously a T vote, now it is not
            T_sample.append(-1.0)
            if spv_cvr == 6: # this was previously a t vote, now it is not
                t_sample.append(-1.0)
            else:
                t_sample.append(0.0)
        elif fpv_bal == 1: # this was previously not a T vote, now it is
            T_sample.append(1.0)
            if spv_bal == 6: # this was previously not a t vote, now it is
                t_sample.append(1.0)
            else:
                t_sample.append(0.0)
        else: # this was previously not a FPV 1 vote, and it still isn't
            T_sample.append(0.0)
            t_sample.append(0.0)
        if fpv_cvr == 6: # this was previously a g vote, now it is not
            g_sample.append(-1.0)
        elif fpv_bal == 6: # this was previously not a g vote, now it is
            g_sample.append(1.0)
        else: 
            g_sample.append(0.0)

m=2
n=3070
gt_sample = [g_sample[i] + t_sample[i] for i in range(len(g_sample))]
Tt_sample = [(m+1)*T_sample[i] - t_sample[i] for i in range(len(t_sample))]
gt_ci_lo, gt_ci_hi = Wald_CI(gt_sample, N=len(BAL), bounds=(-2.0, 2.0))
gt_ci_lo *= len(BAL)
gt_ci_hi *= len(BAL)
gt_ci_lo += 750
gt_ci_hi += 750
print(f"(g+t) CI: ({gt_ci_lo:.4f}, {gt_ci_hi:.4f})")
Tt_ci_lo, Tt_ci_hi = Wald_CI(Tt_sample, N=len(BAL), bounds=(-2.0, 2.0))
Tt_ci_lo *= len(BAL)
Tt_ci_hi *= len(BAL)
Tt_ci_lo += (m+1)*1350 - 0
Tt_ci_hi += (m+1)*1350 - 0
print(f"(m+1)T - t CI: ({Tt_ci_lo:.4f}, {Tt_ci_hi:.4f})")
num_ground = n + (m+1)*10**(-6)
num_hi = num_ground - gt_ci_lo
num_lo = num_ground - gt_ci_hi
k_hi=num_hi/Tt_ci_lo
k_lo=num_lo/Tt_ci_hi
print(f"k CI: ({k_lo:.6f}, {k_hi:.6f})")

# repeat the degree 1 shenangians for step 4

round4_deg0_samples = {i: [] for i in range(2, 7)} 
round4_deg1_samples = {i: [] for i in range(2, 7)} 

for i in range(0, sample_size):
    bal, cvr = shuffled_BAL[i], shuffled_CVR[i]
    fpv_bal, fpv_cvr = project_FPV(bal, hopeful), project_FPV(cvr, hopeful)
    spv_bal, spv_cvr = project_FPV(bal, hopeful_without1), project_FPV(cvr, hopeful_without1)
    if fpv_bal == fpv_cvr:
        if fpv_bal != 1 or spv_bal == spv_cvr:
            for i in range(2,7):
                round4_deg0_samples[i].append(0.0)
                round4_deg1_samples[i].append(0.0)
        else: # both fpv are 1 and spv differ
            # in either case the degree 0 impact null
            for i in range(2,7):
                round4_deg0_samples[i].append(0.0)
                # the deg 1 impact on spv_cvr is -1, on spv_bal is +1, rest are 0
                if i == spv_cvr:
                    round4_deg1_samples[i].append(-1.0)
                elif i == spv_bal:
                    round4_deg1_samples[i].append(1.0)
                else:
                    round4_deg1_samples[i].append(0.0)
    else: # different fpv 
        if fpv_cvr == 1: # previously this was a deg 1 vote, now it is not
            round4_deg1_samples[spv_cvr].append(-1.0)
            round4_deg0_samples[fpv_bal].append(1.0)
            # rest are 0
            for i in range(2,7):
                if i != spv_cvr:
                    round4_deg1_samples[i].append(0.0)
                if i != fpv_bal:
                    round4_deg0_samples[i].append(0.0)
        elif fpv_bal == 1: # previously this was not a deg 1 vote, now it is
            round4_deg1_samples[spv_bal].append(1.0)
            round4_deg0_samples[fpv_cvr].append(-1.0)
            # rest are 0
            for i in range(2,7):
                if i != spv_bal:
                    round4_deg1_samples[i].append(0.0)
                if i != fpv_cvr:
                    round4_deg0_samples[i].append(0.0)
        else: # only degree 0 impact
            round4_deg0_samples[fpv_bal].append(1.0)
            round4_deg0_samples[fpv_cvr].append(-1.0)
            for i in range(2,7):
                round4_deg1_samples[i].append(0.0)
                if i != fpv_bal and i != fpv_cvr:
                    round4_deg0_samples[i].append(0.0)

case4_CVR_FPV = {i: sum(1 for bal in CVR if project_FPV(bal, hopeful_without1) == i) for i in range(2,7)}

conf_intervals_4 = {}
for i in [2,3]:
    deg0_ci_lo, deg0_ci_hi = Wald_CI(round4_deg0_samples[i], N=len(BAL), bounds=(-1.0, 1.0))
    deg0_ci_lo *= len(BAL)
    deg0_ci_hi *= len(BAL)
    deg0_ci_lo += round2_CVR_tallies[i]
    deg0_ci_hi += round2_CVR_tallies[i]
    deg1_ci_lo, deg1_ci_hi = Wald_CI(round4_deg1_samples[i], N=len(BAL), bounds=(-1.0, 1.0)) # in this example, the deg1 effects are non-negative
    deg1_ci_lo *= len(BAL)
    deg1_ci_hi *= len(BAL)
    if i == 2: 
        deg1_ci_lo += 700
        deg1_ci_hi += 700
    total_ci_lo = deg0_ci_lo + (1-k_hi)*deg1_ci_lo
    total_ci_hi = deg0_ci_hi + (1-k_lo)*deg1_ci_hi
    print(f"Round 4, Candidate {i}, Total: CI=({total_ci_lo:.4f}, {total_ci_hi:.4f})")
    conf_intervals_4[i] = (total_ci_lo, total_ci_hi)

if conf_intervals_4[3][1] < conf_intervals_4[2][0]:
    print(f"Step 4 successful: 2 beats 3 head-to-head.")

Discrepancy found in ballot 126: BAL=(6,), CVR=(1, 2)
Discrepancy found in ballot 207: BAL=(6,), CVR=(1, 2)
Round 1, Candidate 1: CI=(652.6351, 706.4315)
Round 1, Candidate 2: CI=(737.0013, 802.9987)
Round 1, Candidate 3: CI=(817.0013, 882.9987)
Round 1, Candidate 4: CI=(317.0013, 382.9987)
Round 1, Candidate 5: CI=(267.0013, 332.9987)
Margin CI (Cand 4 - Cand 5): (-32.9987, 32.9987)
Step 1 successful according to margin audit
Round 2, Candidate 1: CI=(952.6351, 1006.4315)
Round 2, Candidate 2: CI=(737.0013, 802.9987)
Round 2, Candidate 3: CI=(817.0013, 882.9987)
Round 2, Candidate 4: CI=(317.0013, 382.9987)
exhausted in round 2 CI=(93.5685, 147.3649)
q_low = 974.2117, q_high = 992.1438
Step 2 successful: only viable winner is Candidate 1 (reality 2a is viable).
Step 2 successful: only viable loser is Candidate 4 (reality 2b is viable).
(g+t) CI: (93.5685, 147.3649)
(m+1)T - t CI: (2857.9054, 3019.2946)
k CI: (0.967986, 1.041473)
Round 3a, Candidate 2, Total: CI=(737.0013, 803.2046)
Ro

In [ ]:
deg1_ci

1350

In [103]:
# Wilson approach
from statistics import NormalDist
from typing import Iterable, Optional, Tuple

def wilson_mu_ci(sample: Iterable[int], N: Optional[int] = None, alpha: float = 0.05) -> Tuple[float, float]:
    """
    Wilson/Agresti–Coull-style (score-based) CI for μ = p(+1) − p(−1) from a sample of B_i ∈ {−1,0,1}.
    If N is provided, applies the finite-population correction for SRSWOR.

    Returns (lo, hi) on the mean scale μ ∈ [−1, 1].
    """
    x = list(sample)
    n = len(x)
    if n == 0:
        raise ValueError("Sample is empty.")
    if N is not None and not (1 <= n <= N):
        raise ValueError("Require 1 ≤ n ≤ N when N is provided.")

    # Counts
    n_plus  = sum(1 for v in x if v ==  1)
    n_minus = sum(1 for v in x if v == -1)
    n_zero  = n - n_plus - n_minus  # not used directly, but here for clarity

    # z critical and "plus z^2/3 to each cell" adjustment
    z  = NormalDist().inv_cdf(1 - alpha/2)
    z2 = z * z
    n_star = n + z2
    add = z2 / 3.0

    # Adjusted category proportions
    p_plus  = (n_plus  + add) / n_star
    p_minus = (n_minus + add) / n_star
    # p_zero = (n_zero + add) / n_star  # not needed for μ, but implied

    # Wilson center and variance for μ = p_plus - p_minus
    mu_tilde = p_plus - p_minus
    q_tilde  = p_plus + p_minus
    var_mu   = max(0.0, (q_tilde - mu_tilde * mu_tilde) / n_star)

    # Finite population correction (if N provided)
    fpc = (1.0 - n / N) ** 0.5 if N is not None else 1.0

    half = z * (var_mu ** 0.5) * fpc
    lo = max(-1.0, mu_tilde - half)
    hi = min(1.0, mu_tilde + half)
    return (lo, hi)


In [104]:
# shuffle BAL and CVR in the same way
combined = list(zip(BAL, CVR))
random.shuffle(combined)
shuffled_BAL, shuffled_CVR = zip(*combined)
sample_size = 300

#build wald CIs for all 5 candidates in the first round
round1_samples = {i: [] for i in range(1, 7)} # candidate number -> list of sample projections

hopeful = {1,2,3,4,5} # Jill Stein is not hopeful

for i in range(0, sample_size):
    bal, cvr = shuffled_BAL[i], shuffled_CVR[i]
    if bal != cvr:
        print(f"Discrepancy found in ballot {i}: BAL={bal}, CVR={cvr}")
    fpv_bal, fpv_cvr = project_FPV(bal, hopeful), project_FPV(cvr, hopeful)
    if fpv_bal == fpv_cvr:
        for i in range(1,7):
            round1_samples[i].append(0.0)
    else: # fpv_cvr just lost a vote and fpv_bal just gained a vote
        round1_samples[fpv_bal].append(1.0)
        round1_samples[fpv_cvr].append(-1.0)
        for i in range(1,7):
            if i != fpv_bal and i != fpv_cvr:
                round1_samples[i].append(0.0)

CVR_tallies = {
    1: sum(1 for bal in CVR if project_FPV(bal, hopeful) == 1),
    2: sum(1 for bal in CVR if project_FPV(bal, hopeful) == 2),
    3: sum(1 for bal in CVR if project_FPV(bal, hopeful) == 3),
    4: sum(1 for bal in CVR if project_FPV(bal, hopeful) == 4),
    5: sum(1 for bal in CVR if project_FPV(bal, hopeful) == 5),
    6: sum(1 for bal in CVR if project_FPV(bal, hopeful) == 6),
}

for key, value in round1_samples.items():
    ci_lo, ci_hi = wilson_mu_ci(value, N=len(BAL), alpha=0.05)
    ci_lo *= len(BAL)
    ci_hi *= len(BAL)
    ci_lo += CVR_tallies[key]
    ci_hi += CVR_tallies[key]
    print(f"Candidate {key}: CI=({ci_lo:.4f}, {ci_hi:.4f})")
    if key == 6:
        print(f"q_low = {(len(BAL)- ci_hi)/3+10**(-6):.4f}")

Discrepancy found in ballot 182: BAL=(6,), CVR=(2,)
Discrepancy found in ballot 240: BAL=(4, 1), CVR=(4, 1, 5)
Candidate 1: CI=(673.8968, 734.1032)
Candidate 2: CI=(716.4152, 787.3769)
Candidate 3: CI=(815.8968, 876.1032)
Candidate 4: CI=(321.8968, 382.1032)
Candidate 5: CI=(271.8968, 332.1032)
Candidate 6: CI=(78.6231, 149.5848)
q_low = 973.4717
